- このノートブックの内容は2024年5月時点の [LangChain 公式ドキュメント](https://python.langchain.com/v0.1/docs/get_started/introduction)に依拠しています。
- 事前に社内規定PDFを概要欄リンクからダウンロードし、動画内の説明に従ってアップロードしておいてください。

## 準備：API キーの設定

In [6]:
!pip install -q openai
!pip install -q pypdf
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q langchain-chroma
!pip install -q docx2txt
!pip install -q langchain_community
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


In [ ]:

from google.colab import userdata
import openai

# OPENAI_API_KEY を取得
openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = openai.api_key

# RAG（LLM の知識拡張）
RAG を活用することで、LLM は学習データに含まれない知識を得ることができます。

参照用データとしては、現在 LangChain は以下に対応しています：
- .txt
- .csv
- .html
- .json
- .md
- .pdf
- MS Office ファイル

今回は PDF からの知識抽出を試してみましょう。

In [ ]:
#from langchain_community.document_loaders import PyPDFDirectoryLoader
#from langchain_community.document_loaders import UnstructuredWordDocumentLoader
#from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
#loader = Docx2txtLoader("docx/example.docx")
loader = UnstructuredExcelLoader("excel/example.xlsx", mode="elements")
data = loader.load()
data[0]


In [ ]:
data

## 文埋め込みと類似文書検索

In [10]:
from langchain_openai import OpenAIEmbeddings

In [11]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [12]:
from langchain_chroma import Chroma

In [14]:
from langchain_community.vectorstores.utils import filter_complex_metadata
data = filter_complex_metadata(data)
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

In [15]:
# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":1})

In [ ]:
query = "新旧比較テストは？"

retriever.invoke(query)

## Retrieval を活用したチャットモデルの制御

### まずはただ聞いてみる

In [17]:
from langchain_openai import ChatOpenAI

In [ ]:
gpt4o = ChatOpenAI(model="gpt-4o", temperature=0.1)
out = gpt4o.invoke("新旧比較テストは？")
print(out.content)

### 準備：プロンプトテンプレートを作る

In [28]:
from langchain_core.prompts.prompt import PromptTemplate

In [29]:
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="以下を参照して、質問に答えてください。\n\n{context}\n\n質問：{question}"
)

In [30]:
example = {"question":"これは質問です", "context":"これは外部知識です"}

prompt_template.invoke(example)

StringPromptValue(text='以下を参照して、質問に答えてください。\n\nこれは外部知識です\n\n質問：これは質問です')

### 準備：抽出器を作る

In [19]:
# ベクトルデータベースを作る
db = Chroma.from_documents(data, embeddings_model)

# 抽出器を作る
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [ ]:
retriever.invoke("新旧比較テストの進捗は？")

In [ ]:
retrieved = retriever.invoke("VESPA お問合せの内容は？")

print(retrieved[0].page_content)

### 準備：フォーマッターを作る
抽出器の出力を生テキストの形に成形する関数を用意しておく。

In [24]:
def text_formatter(retriever_output):
  raw_text = retriever_output[0].page_content
  raw_text_wo_newline = raw_text.replace("\n", "")
  return raw_text_wo_newline

In [25]:
text_formatter(retrieved)

'大項目中項目小項目ステータス開始日時終了日時担当コメントXXXアプリ名\\nPMO支援資料作成【COSMOS】\\n移行リハーサル計画作成12023-10-20 00:00:002023-10-25 00:00:00顧客RV待ちInitiateAS-IS分析_システム構成情報の可視化(ファイル一覧作成)0.82023-10-12 00:00:002023-10-27 00:00:00オンスケInitiateAS-IS分析_システム構成情報の可視化(機能一覧作成)0.62023-10-16 00:00:002023-10-27 00:00:00遅延（遅延原因：XXX）'

### 抽出・補完・生成のチェインを作る

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [22]:
llm = ChatOpenAI(model="gpt-4o")

retriever、prompt_template、llmが用意できたので、これらを組み合わせて、情報の流れを定義する。LangChainでは、これから書いていくような「インプットからアウトプットまでの流れ」をチェイン（chain）と呼ぶ。

In [31]:

chain = ({"question":RunnablePassthrough(), "context":retriever|text_formatter}
         | prompt_template
         | llm
         )

In [32]:
chain.invoke("新旧比較テストの進捗は？")

AIMessage(content='新旧比較テストの進捗は以下の通りです：\n\n- **画面系比較**：完了。\n- **バッチ系比較**：作業中。詳細確認に時間がかかっており、5月31日に完了見込みです。\n\n全体としては「テスト実施」が進行中であり、一部遅延が発生しています。', response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 748, 'total_tokens': 833}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b27428df-29a4-465c-9ec1-2a2efee72324-0', usage_metadata={'input_tokens': 748, 'output_tokens': 85, 'total_tokens': 833})

In [33]:
resp=chain.invoke("全体的に問題ポイントを教えて")

In [34]:
resp.content

'このプロジェクトの状況を見ると、以下の問題ポイントが浮かび上がります。\n\n1. **移行リハーサル計画作成における顧客レビュー待ち**\n   - ステータスが「顧客RV待ち」となっており、開始日は2023-10-20ですが、終了予定日が2023-10-25と迫っています。顧客レビューが待たれているため、今後の進捗に影響が出る可能性があります。特に顧客からのフィードバックが遅れると、予定通りに次のステップに進めないリスクがあるため、早急にフォローアップが必要です。\n\n2. **AS-IS分析_システム構成情報の可視化(機能一覧作成)の遅延**\n   - このタスクは遅延しており、遅延の原因が「XXX」と記載されています。具体的な遅延原因が明記されていないため、原因の特定と対応策の策定が急務です。遅延が他のタスクに波及しないよう、早急な対応が求められます。\n\n3. **スケジュール管理の必要性**\n   - 全体的にスケジュールがタイトで、特にAS-IS分析の終了予定日が2023-10-27と近接しています。タスクの進捗状況を綿密にチェックし、遅延が他のタスクに波及しないように管理する必要があります。\n\n### 対応策\n- **顧客レビュー待ちのタスクについて**\n  - 顧客とコミュニケーションを取り、レビューの進捗状況を確認し、必要に応じてフォローアップミーティングを設定します。\n  \n- **遅延タスクについて**\n  - 遅延の具体的な原因を早急に特定し、対応策を講じます。例えば、リソースの追加やタスクの優先順位の見直しなどが考えられます。\n\n- **全体スケジュールの再確認**\n  - 全体のスケジュールを再確認し、リスク管理を強化します。定期的な進捗確認ミーティングを設定し、問題が早期に発見できるようにします。\n\nこれらの対応策を講じることで、プロジェクトの進捗を確実に管理し、予定通りの完了を目指すことができます。'